In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
zomato_real= pd.read_csv("/content/drive/MyDrive/Momds/zomato.csv")

In [ ]:
zomato_real.head()

url  \
0  https://www.zomato.com/bangalore/jalsa-banasha...   
1  https://www.zomato.com/bangalore/spice-elephan...   
2  https://www.zomato.com/SanchurroBangalore?cont...   
3  https://www.zomato.com/bangalore/addhuri-udupi...   
4  https://www.zomato.com/bangalore/grand-village...   

                                             address                   name  \
0  942, 21st Main Road, 2nd Stage, Banashankari, ...                  Jalsa   
1  2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...         Spice Elephant   
2  1112, Next to KIMS Medical College, 17th Cross...        San Churro Cafe   
3  1st Floor, Annakuteera, 3rd Stage, Banashankar...  Addhuri Udupi Bhojana   
4  10, 3rd Floor, Lakshmi Associates, Gandhi Baza...          Grand Village   

  online_order book_table   rate  votes                             phone  \
0          Yes        Yes  4.1/5    775    080 42297555\r\n+91 9743772233   
1          Yes         No  4.1/5    787                      080 41714161   
2          Yes         No  3.8/5    918                    +91 9663487993   
3           No         No  3.7/5     88                    +91 9620009302   
4           No         No  3.8/5    166  +91 8026612447\r\n+91 9901210005   

       location            rest_type  \
0  Banashankari        Casual Dining   
1  Banashankari        Casual Dining   
2  Banashankari  Cafe, Casual Dining   
3  Banashankari          Quick Bites   
4  Basavanagudi        Casual Dining   

                                          dish_liked  \
0  Pasta, Lunch Buffet, Masala Papad, Paneer Laja...   
1  Momos, Lunch Buffet, Chocolate Nirvana, Thai G...   
2  Churros, Cannelloni, Minestrone Soup, Hot Choc...   
3                                        Masala Dosa   
4                                Panipuri, Gol Gappe   

                         cuisines approx_cost(for two people)  \
0  North Indian, Mughlai, Chinese                         800   
1     Chinese, North Indian, Thai                         800   
2          Cafe, Mexican, Italian                         800   
3      South Indian, North Indian                         300   
4        North Indian, Rajasthani                         600   

                                        reviews_list menu_item  \
0  [('Rated 4.0', 'RATED\n  A beautiful place to ...        []   
1  [('Rated 4.0', 'RATED\n  Had been here for din...        []   
2  [('Rated 3.0', "RATED\n  Ambience is not that ...        []   
3  [('Rated 4.0', "RATED\n  Great food and proper...        []   
4  [('Rated 4.0', 'RATED\n  Very good restaurant ...        []   

  listed_in(type) listed_in(city)  
0          Buffet    Banashankari  
1          Buffet    Banashankari  
2          Buffet    Banashankari  
3          Buffet    Banashankari  
4          Buffet    Banashankari

# **Data cleaning and Feature Engineering**

In [ ]:
#Deleting Unnnecessary Columns
zomato=zomato_real.drop(['url','dish_liked','phone'],axis=1)
 #Dropping the column "dish_liked", "phone", "url" and saving the new dataset as "zomato"
zomato.head()

address                   name  \
0  942, 21st Main Road, 2nd Stage, Banashankari, ...                  Jalsa   
1  2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...         Spice Elephant   
2  1112, Next to KIMS Medical College, 17th Cross...        San Churro Cafe   
3  1st Floor, Annakuteera, 3rd Stage, Banashankar...  Addhuri Udupi Bhojana   
4  10, 3rd Floor, Lakshmi Associates, Gandhi Baza...          Grand Village   

  online_order book_table   rate  votes      location            rest_type  \
0          Yes        Yes  4.1/5    775  Banashankari        Casual Dining   
1          Yes         No  4.1/5    787  Banashankari        Casual Dining   
2          Yes         No  3.8/5    918  Banashankari  Cafe, Casual Dining   
3           No         No  3.7/5     88  Banashankari          Quick Bites   
4           No         No  3.8/5    166  Basavanagudi        Casual Dining   

                         cuisines approx_cost(for two people)  \
0  North Indian, Mughlai, Chinese                         800   
1     Chinese, North Indian, Thai                         800   
2          Cafe, Mexican, Italian                         800   
3      South Indian, North Indian                         300   
4        North Indian, Rajasthani                         600   

                                        reviews_list menu_item  \
0  [('Rated 4.0', 'RATED\n  A beautiful place to ...        []   
1  [('Rated 4.0', 'RATED\n  Had been here for din...        []   
2  [('Rated 3.0', "RATED\n  Ambience is not that ...        []   
3  [('Rated 4.0', "RATED\n  Great food and proper...        []   
4  [('Rated 4.0', 'RATED\n  Very good restaurant ...        []   

  listed_in(type) listed_in(city)  
0          Buffet    Banashankari  
1          Buffet    Banashankari  
2          Buffet    Banashankari  
3          Buffet    Banashankari  
4          Buffet    Banashankari

In [ ]:
#Removing the Duplicates
zomato.duplicated().sum()
zomato.drop_duplicates(inplace=True)

In [ ]:
#Remove the NaN values from the dataset
zomato.isnull().sum()
zomato.dropna(how='any',inplace=True)

In [ ]:
#Changing the column names
zomato = zomato.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})

In [ ]:
#Some Transformations
zomato['cost'] = zomato['cost'].astype(str) #Changing the cost to string
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost
zomato['cost'] = zomato['cost'].astype(float)
#Removing '/5' from Rates
zomato = zomato.loc[zomato.rate !='NEW']
zomato = zomato.loc[zomato.rate !='-'].reset_index(drop=True)
remove_slash = lambda x: x.replace('/5', '') if type(x) == np.str else x
zomato.rate = zomato.rate.apply(remove_slash).str.strip().astype('float')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [ ]:
# Adjust the column names
zomato.name = zomato.name.apply(lambda x:x.title())
zomato.online_order.replace(('Yes','No'),(True, False),inplace=True)
zomato.book_table.replace(('Yes','No'),(True, False),inplace=True)

In [ ]:
## Computing Mean Rating
restaurants = list(zomato['name'].unique())
zomato['Mean Rating'] = 0

for i in range(len(restaurants)):
    zomato['Mean Rating'][zomato['name'] == restaurants[i]] = zomato['rate'][zomato['name'] == restaurants[i]].mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
## Lower Casing
zomato["reviews_list"] = zomato["reviews_list"].str.lower()

In [ ]:
## Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_punctuation(text))

In [ ]:
## Removal of Stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stop)
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_stopwords(text))

In [ ]:
## Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_urls(text))

zomato[['reviews_list', 'cuisines']].sample(5)

reviews_list  \
3010   rated 40 ratedn beautiful place much ideal dat...   
31576  rated 20 ratedn chai central small snacks poin...   
23335  rated 40 ratedn outlet hyderbadi biryani house...   
26126                                                      
19146  rated 50 ratedn delicious foodnbest food low p...   

                                            cuisines  
3010   North Indian, Mughlai, Mediterranean, Iranian  
31576                                 Tea, Fast Food  
23335     Biryani, North Indian, Chinese, Hyderabadi  
26126                              Juices, Beverages  
19146                                      Fast Food

In [ ]:
# RESTAURANT NAMES:
restaurant_names = list(zomato['name'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]
# Randomly sample 60% of your dataframe
df_percent = zomato.sample(frac=0.5)

# **TF - IDF**

In [ ]:
df_percent.set_index('name', inplace=True)
indices = pd.Series(df_percent.index)

# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])



In [ ]:
def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top restaurants
    recommend_restaurant = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    
    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new
recommend('Pai Vihar')



TOP 10 RESTAURANTS LIKE Pai Vihar WITH SIMILAR REVIEWS: 


cuisines  \
Myu Bar At Gilly'S Redefined                        American, North Indian, Pizza, Finger Food, Co...   
Sixth Avenue Cafe And Patisserie                                              Cafe, Italian, Desserts   
Zee5 Loft                                                  Cafe, North Indian, Pizza, Sandwich, Salad   
The Only Place                                      Italian, American, Steak, Continental, BBQ, Salad   
The Yogisthaan Cafe                                              Cafe, Healthy Food, Salad, Beverages   
World Of Asia                                                                                   Asian   
Tangerine - Davanam Sarovar Portico Suites          Continental, Italian, North Indian, Asian, Bak...   
The CafãÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ...                                               Cafe   
Puran Da Dhaba - The Pride Hotel                                                         North Indian   
Royal Biryani Kitchen                                                    Biryani, North Indian, Kebab   

                                                    Mean Rating   cost  
Myu Bar At Gilly'S Redefined                               4.41    1.2  
Sixth Avenue Cafe And Patisserie                           4.36  600.0  
Zee5 Loft                                                  4.35  500.0  
The Only Place                                             4.10    1.0  
The Yogisthaan Cafe                                        4.01  600.0  
World Of Asia                                              3.84  300.0  
Tangerine - Davanam Sarovar Portico Suites                 3.84    1.1  
The CafãÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ...         3.84  800.0  
Puran Da Dhaba - The Pride Hotel                           3.83    1.7  
Royal Biryani Kitchen                                      3.72  600.0

# **Jaccard Simillarity**

In [ ]:
x=zomato_real.loc[:,["rest_type","cuisines","approx_cost(for two people)","name"]]
x.head(5)

rest_type                        cuisines  \
0        Casual Dining  North Indian, Mughlai, Chinese   
1        Casual Dining     Chinese, North Indian, Thai   
2  Cafe, Casual Dining          Cafe, Mexican, Italian   
3          Quick Bites      South Indian, North Indian   
4        Casual Dining        North Indian, Rajasthani   

  approx_cost(for two people)                   name  
0                         800                  Jalsa  
1                         800         Spice Elephant  
2                         800        San Churro Cafe  
3                         300  Addhuri Udupi Bhojana  
4                         600          Grand Village

In [ ]:
cuisine = x['cuisines'].tolist()

In [ ]:
print(cuisine)

['North Indian, Mughlai, Chinese', 'Chinese, North Indian, Thai', 'Cafe, Mexican, Italian', 'South Indian, North Indian', 'North Indian, Rajasthani', 'North Indian', 'North Indian, South Indian, Andhra, Chinese', 'Pizza, Cafe, Italian', 'Cafe, Italian, Continental', 'Cafe, Mexican, Italian, Momos, Beverages', 'Cafe', 'Cafe, Italian, Continental', 'Cafe, Chinese, Continental, Italian', 'Cafe, Continental', 'Cafe, Mexican, Italian', 'Cafe', 'Cafe, Fast Food, Continental, Chinese, Momos', 'Chinese, Cafe, Italian', 'Cafe, Italian, American', 'Cafe, Chinese, Continental, Italian', 'Cafe, French, North Indian', 'Cafe, Pizza, Fast Food, Beverages', 'Cafe, Fast Food', 'Italian, Fast Food, Cafe, European', 'Cafe', 'Cafe, Bakery', 'Cafe, South Indian', 'Cafe, Fast Food, Beverages', 'Cafe, Fast Food', 'North Indian, Cafe, Chinese, Fast Food', 'Cafe, Italian', 'North Indian, Fast Food, Chinese, Burger', 'Bakery, Desserts', 'Pizza', 'North Indian, Biryani, Fast Food', 'Biryani', 'North Indian, Chin

In [ ]:
x.cuisines.unique()

array(['North Indian, Mughlai, Chinese', 'Chinese, North Indian, Thai',
       'Cafe, Mexican, Italian', ...,
       'North Indian, Street Food, Biryani', 'Chinese, Mughlai',
       'North Indian, Chinese, Arabian, Momos'], dtype=object)

In [ ]:
x["cuisines"] = x["cuisines"].str.split(", ")
# df display
x.head(5)

rest_type                          cuisines  \
0        Casual Dining  [North Indian, Mughlai, Chinese]   
1        Casual Dining     [Chinese, North Indian, Thai]   
2  Cafe, Casual Dining          [Cafe, Mexican, Italian]   
3          Quick Bites      [South Indian, North Indian]   
4        Casual Dining        [North Indian, Rajasthani]   

  approx_cost(for two people)                   name  
0                         800                  Jalsa  
1                         800         Spice Elephant  
2                         800        San Churro Cafe  
3                         300  Addhuri Udupi Bhojana  
4                         600          Grand Village

In [ ]:
cuisine = x['cuisines'].tolist()

In [ ]:
print(cuisine)

[['North Indian', 'Mughlai', 'Chinese'], ['Chinese', 'North Indian', 'Thai'], ['Cafe', 'Mexican', 'Italian'], ['South Indian', 'North Indian'], ['North Indian', 'Rajasthani'], ['North Indian'], ['North Indian', 'South Indian', 'Andhra', 'Chinese'], ['Pizza', 'Cafe', 'Italian'], ['Cafe', 'Italian', 'Continental'], ['Cafe', 'Mexican', 'Italian', 'Momos', 'Beverages'], ['Cafe'], ['Cafe', 'Italian', 'Continental'], ['Cafe', 'Chinese', 'Continental', 'Italian'], ['Cafe', 'Continental'], ['Cafe', 'Mexican', 'Italian'], ['Cafe'], ['Cafe', 'Fast Food', 'Continental', 'Chinese', 'Momos'], ['Chinese', 'Cafe', 'Italian'], ['Cafe', 'Italian', 'American'], ['Cafe', 'Chinese', 'Continental', 'Italian'], ['Cafe', 'French', 'North Indian'], ['Cafe', 'Pizza', 'Fast Food', 'Beverages'], ['Cafe', 'Fast Food'], ['Italian', 'Fast Food', 'Cafe', 'European'], ['Cafe'], ['Cafe', 'Bakery'], ['Cafe', 'South Indian'], ['Cafe', 'Fast Food', 'Beverages'], ['Cafe', 'Fast Food'], ['North Indian', 'Cafe', 'Chinese', 

Enter Restaurant ID's to compare food items

In [ ]:
n1=int(input("Enter ID1 = "))
n2=int(input("Enter ID2 = "))

Enter ID1 = 20
Enter ID2 = 30


In [ ]:
indices = [n1,n2]
a = x['name'].iloc[indices]
b=x['rest_type'].iloc[indices]
names = a.tolist()
types=b.tolist()
print(names)
print(types)

['The Vintage Cafe', 'Redberrys']
['Cafe', 'Cafe']


In [ ]:
def jaccard(list1, list2):
  if(types[0]==types[1]):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union
#find Jaccard Similarity between the two restaurants 

In [ ]:
print(jaccard(cuisine[n1],cuisine[n2]))

0.25


# **Apriori Algorithm**

In [ ]:
print(x['cuisines'])

0                [North Indian,  Mughlai,  Chinese]
1                   [Chinese,  North Indian,  Thai]
2                        [Cafe,  Mexican,  Italian]
3                     [South Indian,  North Indian]
4                       [North Indian,  Rajasthani]
                            ...                    
51712                                 [Continental]
51713                                 [Finger Food]
51714                                 [Finger Food]
51715                                 [Finger Food]
51716    [Finger Food,  North Indian,  Continental]
Name: cuisines, Length: 51717, dtype: object


In [ ]:
len(x['cuisines'])

51717

In [ ]:
pip install efficient-apriori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
cuisine = cuisine[:400]

In [ ]:
from efficient_apriori import apriori

# our min support is 7, but it has to be expressed as a percentage for efficient-apriori
min_support = 7/len(cuisine)
# min confidence allows you to delete rules with low confidence.
# For now set min_confidence = 0 to obtain all the rules
min_confidence = 0
itemsets, rules = apriori(cuisine,min_support=min_support, min_confidence=min_confidence)

In [ ]:
print(itemsets)

{1: {('North Indian',): 139, ('Chinese',): 125, ('Thai',): 8, ('Cafe',): 50, ('Mexican',): 7, ('Italian',): 36, ('South Indian',): 106, ('Andhra',): 12, ('Pizza',): 22, ('Continental',): 22, ('Momos',): 12, ('Beverages',): 46, ('Fast Food',): 93, ('Bakery',): 16, ('Burger',): 11, ('Desserts',): 37, ('Biryani',): 49, ('Street Food',): 32, ('Rolls',): 17, ('Ice Cream',): 19, ('Salad',): 7, ('Seafood',): 14, ('Kebab',): 9, ('Mithai',): 9, ('Sandwich',): 9, ('Juices',): 16, ('Mangalorean',): 10}, 2: {('Andhra', 'Chinese'): 7, ('Andhra', 'North Indian'): 10, ('Bakery', 'Desserts'): 8, ('Beverages', 'Cafe'): 9, ('Beverages', 'Desserts'): 9, ('Beverages', 'Fast Food'): 15, ('Beverages', 'Ice Cream'): 9, ('Beverages', 'North Indian'): 7, ('Beverages', 'South Indian'): 7, ('Biryani', 'Chinese'): 23, ('Biryani', 'Fast Food'): 7, ('Biryani', 'Kebab'): 7, ('Biryani', 'North Indian'): 26, ('Biryani', 'Seafood'): 7, ('Biryani', 'South Indian'): 13, ('Cafe', 'Chinese'): 10, ('Cafe', 'Continental'): 1

In [ ]:
for rule in rules:
  print(rule)

{Chinese} -> {Andhra} (conf: 0.056, supp: 0.018, lift: 1.867, conv: 1.028)
{Andhra} -> {Chinese} (conf: 0.583, supp: 0.018, lift: 1.867, conv: 1.650)
{North Indian} -> {Andhra} (conf: 0.072, supp: 0.025, lift: 2.398, conv: 1.045)
{Andhra} -> {North Indian} (conf: 0.833, supp: 0.025, lift: 2.398, conv: 3.915)
{Desserts} -> {Bakery} (conf: 0.216, supp: 0.020, lift: 5.405, conv: 1.225)
{Bakery} -> {Desserts} (conf: 0.500, supp: 0.020, lift: 5.405, conv: 1.815)
{Cafe} -> {Beverages} (conf: 0.180, supp: 0.022, lift: 1.565, conv: 1.079)
{Beverages} -> {Cafe} (conf: 0.196, supp: 0.022, lift: 1.565, conv: 1.088)
{Desserts} -> {Beverages} (conf: 0.243, supp: 0.022, lift: 2.115, conv: 1.169)
{Beverages} -> {Desserts} (conf: 0.196, supp: 0.022, lift: 2.115, conv: 1.128)
{Fast Food} -> {Beverages} (conf: 0.161, supp: 0.037, lift: 1.403, conv: 1.055)
{Beverages} -> {Fast Food} (conf: 0.326, supp: 0.037, lift: 1.403, conv: 1.139)
{Ice Cream} -> {Beverages} (conf: 0.474, supp: 0.022, lift: 4.119, con